### **Data Preprocessing**



In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os

In [ ]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "x_accel",
    "y_accel",
    "z_accel",
]

# Output classes to learn how to classify
LABELS = [
    "Walking", 
    "Jogging", 
    "Uptairs", 
    "Downstairs", 
    "Sitting", 
    "Standing"
] 

In [ ]:
from numpy import genfromtxt
import pandas as pd
x = genfromtxt('/content/drive/MyDrive/Colab Notebooks/WISDM_x.csv', delimiter=',')
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WISDM_y.csv').to_numpy()

In [ ]:
def create_series(x,y,timestep,overlap):

  slide_step = int(timestep*(1-overlap))
  data_num = int((len(x)/slide_step)-1)
  
  dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
  labels = list()

  for i in range(data_num):
    labels.append(y[slide_step*(i+1)-1])
    for j in range(timestep):
      dataset[i,j,:] = x[slide_step*i+j,:]

  return dataset,np.array(labels)


In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# y_pred.append("5")
dataset,labels = create_series(x,y,16,0.5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [ ]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep


# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(39321, 16, 3) (39321, 6) 2.873419266367973 6.908464983446615
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


### **Model**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.GRU(256, return_sequences=True,input_shape=(n_steps, n_input)))
model.add(layers.Dropout(0.2))
model.add(layers.GRU(128, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.GRU(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.GRU(32))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(6, activation='softmax'))

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

history = model.fit(X_train,y_train,batch_size=64,epochs=10000,validation_split=0.1,callbacks=[callback])

model.save('/Users/wangshiyu/Desktop/gru_model')


Epoch 1/10000
1291/1291 [==============================] - 27s 15ms/step - loss: 0.4195 - accuracy: 0.8559 - val_loss: 0.2185 - val_accuracy: 0.9263
Epoch 2/10000
1291/1291 [==============================] - 16s 12ms/step - loss: 0.2083 - accuracy: 0.9324 - val_loss: 0.1713 - val_accuracy: 0.9460
Epoch 3/10000
1291/1291 [==============================] - 16s 12ms/step - loss: 0.1577 - accuracy: 0.9493 - val_loss: 0.1657 - val_accuracy: 0.9505
Epoch 4/10000
1291/1291 [==============================] - 16s 12ms/step - loss: 0.1280 - accuracy: 0.9596 - val_loss: 0.1454 - val_accuracy: 0.9560
Epoch 5/10000
1291/1291 [==============================] - 16s 12ms/step - loss: 0.1081 - accuracy: 0.9654 - val_loss: 0.1469 - val_accuracy: 0.9588
Epoch 6/10000
1291/1291 [==============================] - 16s 12ms/step - loss: 0.0921 - accuracy: 0.9716 - val_loss: 0.1742 - val_accuracy: 0.9547
Epoch 7/10000
1291/1291 [==============================] - 16s 13ms/step - loss: 0.0832 - accuracy: 0.9744

In [ ]:
model.evaluate(X_test, y_test)

1229/1229 [==============================] - 7s 6ms/step - loss: 0.1578 - accuracy: 0.9622


[0.1577976644039154, 0.9622339010238647]